# Estimación por dominios
    - Los dominios se consideran subgrupos de la población. Por ejemplo, hombres y mujeres. En la estimación por dominio se toma en cuenta que la muestra obtenida quizá no mantenga la misma proporciónde elementos de cada dominio. Con lo cuál se consigue obtener estimaciones separadas para cada dominio.
    
    - Como ejemplo, una población que cuente con 70% mujeres y 30% hombres puede ser muestreada aleatoriamente y la muestra quizá contenga 50% hombres y mujeres, por lo que las inferencias de cada subgrupo no sea la adecuada. Adicionalmente, considerar cada subgrupo como una población separada, podría no arrojar los resultados correctos
    
    - Se utiliza cuando solo se tiene el marco de la población entera, pero no se tiene conocimiento de los elementos específicos de cada dominio

### Conclusiones
    - Los resultados de aplicar este método no siempre serán mejorar la precisión los estimadores, debido a que se tiene que considerar que al muestrear, no siempre se guardará la misma proporción de elementos en el dominio
        Por ejemplo: El estimador mejora para los hombres (65% de la población), mientras que se vuelve más grande para las mujeres (35% de la población)
        
        -n_d/n se aproxima a N_d/N cuando la muestra es grande. En este caso hacer estimaciones filtrando se aproxima a lo que se obtiene al usar la estimación en dominios

### 1] Definir una población

In [58]:
import pandas as pd
import numpy as np

# Settings
N = 1000
n = 30
P = 0.35
confidence = 0.95

# Define domain according to theoretical proportion
sex = np.where(np.random.uniform(size=N)<P, 1, 0)

# Consider different means deppending on sex
score = np.random.normal(loc=6.5, scale=1.5, size=N) + (sex==0)
money = np.random.normal(loc=200, scale=50, size=N) + 100*(sex==1)

# Put together data into a df
df_population = pd.DataFrame(np.vstack([score, money, sex]).T, columns=["score", "money", "sex"])



# Select a random sample --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 

# Create auxiliary column to randomize the sample
df_population["mask"] = np.random.uniform(size=N)

# Sort and select elements randomly
df_sample = df_population.sort_values(by=["mask"])[:n]
df_sample.drop(["mask"], axis=1, inplace=True)

# Drop auxiliary column
df_population.drop(["mask"], axis=1, inplace=True)

### 2] Estimación sin dominios
    - En la primera estimación no se utiliza el dominio y simplemente se hace un filter para obtener una muestra sin considerar la proporción real de elementos que pertenecen a éste

In [59]:
# Analysis done using female as the domain of interest
from scipy.stats import t

# Estimation by filtering the students in the domain
n_d = df_sample["sex"].sum()
mean = df_sample["score"][df_sample["sex"]==0].mean()
var = df_sample["score"][df_sample["sex"]==0].var(ddof=1)
mean_var = (1-n_d/N) * var/n_d
mean_std = mean_var**0.5
mean_interval = mean + mean_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))

# Results
print("La media real de score en las mujeres es:\n",round(df_population["score"][sex==0].mean(),2))
print("\nLa estimación de la media es\n",round(mean,2))
print("\nCon un intervalo de confianza\n",np.round(mean_interval,2))

La media real de score en las mujeres es:
 7.54

La estimación de la media es
 7.76

Con un intervalo de confianza
 [6.72 8.8 ]


### 3] Estimación por dominios
    - Se define:
        n: tamaño de muestra
        n_d: elementos del dominio dentro de la muestra
        x_i = 1 if el elemento pertenece al dominio else 0
            De otra manera:
            sum(x_i) = n_d
        Y: variable de interés
        
    - Se estima la media del dominio
        mean_d = sum(y_i if i pertenece al dominio else 0) / sum(x_i)
            De otra manera:
            mean_d = sum(y_i) / n_d para los elementos en el dominio

    - La varianza del estimador de media es:
        var_mean_d = (1-n/N) * n^2/(n*n_d^2) * 1/(n-1) * sum( (y_i - mean_d^2) )
        donde:
            S_d^2: varianza de los elementos muestreados en el dominio

In [60]:
# Estimation by filtering the students in the domain
n_d = df_sample["sex"].sum()
mean_d = df_sample["score"][df_sample["sex"]==0].mean()
mean_d_var = (1-n/N) * n**2/(n*n_d**2) * 1/(n-1) * ((df_sample["score"][df_sample["sex"]==0] - mean_d)**2).sum()
mean_d_std = mean_d_var**0.5
mean_d_interval = mean_d + mean_d_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))

# Results
print("La media real de score en las mujeres es:\n",round(df_population["score"][sex==0].mean(),2))
print("\nLa estimación de la media es\n",round(mean_d,2))
print("\nCon un intervalo de confianza\n",np.round(mean_d_interval,2))

La media real de score en las mujeres es:
 7.54

La estimación de la media es
 7.76

Con un intervalo de confianza
 [6.67 8.85]
